In [6]:
import torch
from models import resnet110
# 初始化，并加载预训练权重
torch_model = resnet110(pretrained=True, checkpoint='checkpoint/model_best.pth')

totoal blocks num: 54
0:0.009259259259259259
1:0.018518518518518517
2:0.027777777777777776
3:0.037037037037037035
4:0.046296296296296294
5:0.05555555555555555
6:0.06481481481481481
7:0.07407407407407407
8:0.08333333333333333
9:0.09259259259259259
10:0.10185185185185185
11:0.1111111111111111
12:0.12037037037037036
13:0.12962962962962962
14:0.1388888888888889
15:0.14814814814814814
16:0.1574074074074074
17:0.16666666666666666
18:0.17592592592592593
19:0.18518518518518517
20:0.19444444444444445
21:0.2037037037037037
22:0.21296296296296297
23:0.2222222222222222
24:0.23148148148148148
25:0.24074074074074073
26:0.25
27:0.25925925925925924
28:0.26851851851851855
29:0.2777777777777778
30:0.28703703703703703
31:0.2962962962962963
32:0.3055555555555556
33:0.3148148148148148
34:0.32407407407407407
35:0.3333333333333333
36:0.3425925925925926
37:0.35185185185185186
38:0.3611111111111111
39:0.37037037037037035
40:0.37962962962962965
41:0.3888888888888889
42:0.39814814814814814
43:0.4074074074074074


In [9]:
import torch
from models_paddle import resnet110
# 初始化，并加载预训练权重
paddle_model = resnet110(checkpoint=None)

totoal blocks num: 54
0:0.009259259259259259
1:0.018518518518518517
2:0.027777777777777776
3:0.037037037037037035
4:0.046296296296296294
5:0.05555555555555555
6:0.06481481481481481
7:0.07407407407407407
8:0.08333333333333333
9:0.09259259259259259
10:0.10185185185185185
11:0.1111111111111111
12:0.12037037037037036
13:0.12962962962962962
14:0.1388888888888889
15:0.14814814814814814
16:0.1574074074074074
17:0.16666666666666666
18:0.17592592592592593
19:0.18518518518518517
20:0.19444444444444445
21:0.2037037037037037
22:0.21296296296296297
23:0.2222222222222222
24:0.23148148148148148
25:0.24074074074074073
26:0.25
27:0.25925925925925924
28:0.26851851851851855
29:0.2777777777777778
30:0.28703703703703703
31:0.2962962962962963
32:0.3055555555555556
33:0.3148148148148148
34:0.32407407407407407
35:0.3333333333333333
36:0.3425925925925926
37:0.35185185185185186
38:0.3611111111111111
39:0.37037037037037035
40:0.37962962962962965
41:0.3888888888888889
42:0.39814814814814814
43:0.4074074074074074


In [10]:
#权重转换函数
from collections import OrderedDict

def load_pytorch_pretrain_model(paddle_model, pytorch_state_dict):

    paddle_weight=paddle_model.state_dict()
    print("paddle num_params:",len(paddle_weight))
    print("torch num_params:", len(pytorch_state_dict))
    new_weight_dict=OrderedDict()

    torch_key_list=[]
    for key in pytorch_state_dict.keys():
        if "num_batches_tracked" in key:
            continue
        torch_key_list.append(key)

    for torch_key, paddle_key in zip(torch_key_list, paddle_weight.keys()):
        print(torch_key, paddle_key, pytorch_state_dict[torch_key].shape,paddle_weight[paddle_key].shape)
        if len(pytorch_state_dict[torch_key].shape) == 0:
            continue
        ##handle all FC weight cases
        if ("fc" in torch_key and "weight" in torch_key) or (len(pytorch_state_dict[torch_key].shape)==2 and pytorch_state_dict[torch_key].shape[0] == pytorch_state_dict[torch_key].shape[1]):
            new_weight_dict[paddle_key] = pytorch_state_dict[torch_key].cpu().detach().numpy().T.astype("float32")
        elif int(paddle_weight[paddle_key].shape[-1])==int(pytorch_state_dict[torch_key].shape[-1])  :
            new_weight_dict[paddle_key]=pytorch_state_dict[torch_key].cpu().detach().numpy().astype("float32")
        else:
            new_weight_dict[paddle_key] = pytorch_state_dict[torch_key].cpu().detach().numpy().T.astype("float32")
    paddle_model.set_dict(new_weight_dict)
    return paddle_model.state_dict()

In [13]:
#实现权重转换
pytorch_state_dict = torch_model.state_dict()
paddle_state_dict = load_pytorch_pretrain_model(paddle_model, pytorch_state_dict)

paddle num_params: 547
torch num_params: 656
conv1.weight conv1.weight torch.Size([16, 3, 3, 3]) [16, 3, 3, 3]
bn1.weight bn1.weight torch.Size([16]) [16]
bn1.bias bn1.bias torch.Size([16]) [16]
bn1.running_mean bn1._mean torch.Size([16]) [16]
bn1.running_var bn1._variance torch.Size([16]) [16]
layer1.0.conv1.weight layer1.0.conv1.weight torch.Size([16, 16, 3, 3]) [16, 16, 3, 3]
layer1.0.bn1.weight layer1.0.bn1.weight torch.Size([16]) [16]
layer1.0.bn1.bias layer1.0.bn1.bias torch.Size([16]) [16]
layer1.0.bn1.running_mean layer1.0.bn1._mean torch.Size([16]) [16]
layer1.0.bn1.running_var layer1.0.bn1._variance torch.Size([16]) [16]
layer1.0.conv2.weight layer1.0.conv2.weight torch.Size([16, 16, 3, 3]) [16, 16, 3, 3]
layer1.0.bn2.weight layer1.0.bn2.weight torch.Size([16]) [16]
layer1.0.bn2.bias layer1.0.bn2.bias torch.Size([16]) [16]
layer1.0.bn2.running_mean layer1.0.bn2._mean torch.Size([16]) [16]
layer1.0.bn2.running_var layer1.0.bn2._variance torch.Size([16]) [16]
layer1.1.conv1.wei

layer3.0.bn2.running_mean layer3.0.bn2._mean torch.Size([64]) [64]
layer3.0.bn2.running_var layer3.0.bn2._variance torch.Size([64]) [64]
layer3.1.conv1.weight layer3.1.conv1.weight torch.Size([64, 64, 3, 3]) [64, 64, 3, 3]
layer3.1.bn1.weight layer3.1.bn1.weight torch.Size([64]) [64]
layer3.1.bn1.bias layer3.1.bn1.bias torch.Size([64]) [64]
layer3.1.bn1.running_mean layer3.1.bn1._mean torch.Size([64]) [64]
layer3.1.bn1.running_var layer3.1.bn1._variance torch.Size([64]) [64]
layer3.1.conv2.weight layer3.1.conv2.weight torch.Size([64, 64, 3, 3]) [64, 64, 3, 3]
layer3.1.bn2.weight layer3.1.bn2.weight torch.Size([64]) [64]
layer3.1.bn2.bias layer3.1.bn2.bias torch.Size([64]) [64]
layer3.1.bn2.running_mean layer3.1.bn2._mean torch.Size([64]) [64]
layer3.1.bn2.running_var layer3.1.bn2._variance torch.Size([64]) [64]
layer3.2.conv1.weight layer3.2.conv1.weight torch.Size([64, 64, 3, 3]) [64, 64, 3, 3]
layer3.2.bn1.weight layer3.2.bn1.weight torch.Size([64]) [64]
layer3.2.bn1.bias layer3.2.bn

In [15]:
# 保存模型权重
import paddle
paddle.save(paddle_state_dict, "checkpoint/model_paddle.pdparams")

In [17]:
# 得到pytorch版本模型输出结果
import numpy as np
torch_model.eval()
img = np.ones([1,3,32,32]).astype("float32")
img =torch.from_numpy(img)
outputs_t = torch_model(img)
print(outputs_t)

tensor([[ 1.9227, -3.1553,  2.9445,  2.7719, -1.4646,  1.6979,  3.8195, -3.4764,
         -1.6798, -3.3795]], grad_fn=<AddmmBackward>)


In [18]:
# 得到paddle版本模型输出结果
paddle_state_dict = paddle.load("checkpoint/model_paddle.pdparams")
paddle_model.set_state_dict(paddle_state_dict)
paddle_model.eval()
img = np.ones([1,3,32,32]).astype('float32')
img = paddle.to_tensor(img)
outputs_p = paddle_model(img)
print (outputs_p)

Tensor(shape=[1, 10], dtype=float32, place=CPUPlace, stop_gradient=False,
       [[ 1.92266512, -3.15525961,  2.94448066,  2.77187514, -1.46463239,  1.69787252,  3.81953859, -3.47642469, -1.67979658, -3.37954140]])


In [19]:
# 判断是否输出大致一致
np.testing.assert_allclose(outputs_p.detach().numpy(), outputs_t.detach().numpy(), atol=8e-4)#默认rtol(相对容忍度)为1e-7，设绝对容忍度为8e-4
print("outputs_p is equal to outputs_t")

outputs_p is equal to outputs_t


In [21]:
# 在cifar10 测试集上评估，已知pytorch 模型的test acc为95.03%
%cd ../DNSD/
!python eval.py --checkpoint checkpoint/model_paddle.pdparams

/home/lab-zhai.pucheng/code1/DNSD
Namespace(checkpoint='checkpoint/model_paddle.pdparams', data_dir='data', dataset='cifar10', drop_path_rate=0.5, epochs=500, gamma=0.1, high_level_api=False, learning_rate=0.1, milestones=[250, 375], mode='train', momentum=0.9, nesterov=True, net='resnet110', optimizer='sgd', save_best=False, save_dir='output', save_interval=10, test_batch_size=128, train_batch_size=128, weight_decay=0.0001)
totoal blocks num: 54
0:0.009259259259259259
1:0.018518518518518517
2:0.027777777777777776
3:0.037037037037037035
4:0.046296296296296294
5:0.05555555555555555
6:0.06481481481481481
7:0.07407407407407407
8:0.08333333333333333
9:0.09259259259259259
10:0.10185185185185185
11:0.1111111111111111
12:0.12037037037037036
13:0.12962962962962962
14:0.1388888888888889
15:0.14814814814814814
16:0.1574074074074074
17:0.16666666666666666
18:0.17592592592592593
19:0.18518518518518517
20:0.19444444444444445
21:0.2037037037037037
22:0.21296296296296297
23:0.2222222222222222
24:0.23

In [22]:
# 由此可见，模型转换基本没有问题！